In [ ]:
#!/usr/bin/env python3
import sys, os
import json
import numpy as np
import argparse
from parse import parse
import pandas as pd
import h5py
import time
#!pip install ipympl==0.7.0 --user

import matplotlib.pyplot as plt
plt.style.use('../clint.mpl')
%matplotlib widget

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
from pygama.io.daq_to_raw import daq_to_raw

In [ ]:
daq_dir = '/global/cfs/cdirs/legend/data/sisadcnl/Data'
raw_dir = '/global/cfs/cdirs/legend/data/sisadcnl/LH5/raw'

# Make Raw Files

In [ ]:
config = {'daq': 'ORCA'}
files = {
    1: ['2021/07/2021-7-28-CAGERun', np.arange(2,3)],
    2: ['2021/08/2021-8-2-CAGERun', np.arange(3,16)], #run 16 is bad/corrupted
    3: ['2021/08/2021-8-9-CAGERun', np.arange(17,37)],
    4: ['2021/08/2021-8-10-CAGERun', np.arange(37,65)]
}

runs = np.arange(1,5)
for r in runs:
    print(files[r])
    prefix = files[r][0]
    cycles = files[r][1]
    for cyc in cycles:
        f_daq = f'{daq_dir}/{prefix}{cyc}' 
        f_raw = f'{raw_dir}/run{cyc}.lh5'
        if not os.path.exists(f_raw):
            daq_to_raw(f_daq, f_raw, config=config)
        else:
            print(f'Raw file already exists for cycle {cyc}')

# Plot Individual Waveforms

In [ ]:
def plot_wf(run, ind, save=False, name=None):
    plt.figure()
    with h5py.File(f'{raw_dir}/run{run}.lh5') as f:
        values = f['ORSIS3302DecoderForEnergy']['raw']['waveform']['values'][ind]
        samples = np.arange(len(values))
        plt.plot(samples, values)
        plt.xlabel('Sample')
        plt.ylabel('ADU')
    if save:
        name = name if name else 'adcnl_wf_run{run}_{ind}.png'
        plt.savefig(name)
    plt.show()

In [ ]:
plot_wf(6, 3000)

# Find Low Counts

In [ ]:
def drop_badwfs(values, threshold=6000):
    return np.delete(values, np.unique(np.where(values < threshold)[0]), axis=0)

# Histogram ADCs

In [ ]:
def hist_adcs(runs, bins, plot=True):
    counts = np.zeros(len(bins)-1)
    for r in runs:
        print(f'Getting waveforms from run {r}...')
        with h5py.File(f'{raw_dir}/run{r}.lh5') as f:
            ndval = drop_badwfs(f['ORSIS3302DecoderForEnergy']['raw']['waveform']['values'][()])
            bc = np.bincount(np.ravel(ndval))
            counts += np.concatenate((bc, np.zeros(len(counts)-len(bc))))
    if plot:
        plt.figure()
        pgh.plot_hist(counts, bins)
    return counts

In [ ]:
bins = np.arange(-0.5, 64001.5)

In [ ]:
counts_up = hist_adcs(np.arange(37,65), bins, False)

In [ ]:
counts_overnight = counts_up
counts_up = hist_adcs(np.arange(2,10), bins, False)
counts_down = hist_adcs(np.arange(10,16), bins, False)


In [ ]:
counts = counts_overnight + counts_up

In [ ]:
lo = 6400
hi = 57800
dnlplus_overnight = counts_overnight/np.mean(counts_overnight[lo:hi])
dnlplus_up = counts_up/np.mean(counts_up[lo:hi])
dnlplus_down = counts_down/np.mean(counts_down[lo:hi])
dnlplus = counts/np.mean(counts[lo:hi])

In [ ]:
inl_overnight = np.cumsum((dnlplus_overnight - 1)[lo:hi])
inl_up = np.cumsum((dnlplus_up - 1)[lo:hi])
inl_down = np.cumsum((dnlplus_down - 1)[lo:hi])
inl = np.cumsum((dnlplus - 1)[lo:hi])

In [ ]:
plt.figure()
pgh.plot_hist(counts_overnight, bins, label='overnight')
pgh.plot_hist(counts_up, bins, label='2.5 hour')
plt.title('Up Ramp')
plt.xlabel('adu')
plt.ylabel('Counts')
plt.legend()

In [ ]:
# DNL plots
plt.figure()
pgh.plot_hist(dnlplus_overnight, bins, label='overnight', alpha=0.6)
plt.title('Up Ramp Comparison')
plt.xlabel('adu')
plt.ylabel('DNL+1')
# plt.figure()
pgh.plot_hist(dnlplus_up, bins, label='2.5 hour', alpha=0.6)
plt.axhline(1, color='red')
# plt.axhline(1, color='orange')
plt.legend()
# plt.title('Up Ramp')
# plt.xlabel('adu')
# plt.ylabel('DNL+1')
plt.figure()
pgh.plot_hist(dnlplus_down, bins)
plt.axhline(1, color='orange')
plt.title('Down Ramp')
plt.xlabel('adu')
plt.ylabel('DNL+1')
plt.figure()
pgh.plot_hist(dnlplus, bins)
plt.axhline(1, color='orange')
plt.title('All Waveforms')
plt.xlabel('adu')
plt.ylabel('DNL+1')

In [ ]:
# INL Plots
plt.figure()
pgh.plot_hist(inl_overnight, bins[lo:hi+1])
plt.title('Overnight Ramp')
plt.xlabel('adu')
plt.ylabel('INL')
plt.figure()
pgh.plot_hist(inl_up, bins[lo:hi+1])
plt.title('Up Ramp')
plt.xlabel('adu')
plt.ylabel('INL')
plt.figure()
pgh.plot_hist(inl_down, bins[lo:hi+1])
plt.title('Down Ramp')
plt.xlabel('adu')
plt.ylabel('INL')
plt.figure()
pgh.plot_hist(inl, bins[lo:hi+1])
plt.title('All Waveforms')
plt.xlabel('adu')
plt.ylabel('INL')

# Superpulses

In [ ]:
def norm_df(df):
    subbed = df.sub(df.min(axis=1),axis=0)
    normed = subbed.div(subbed.max(axis=1), axis=0)
    return normed

def mean_df(df):
    return df.mean().to_frame().transpose()

def get_superpulse(runs, plot=True):
    sums = np.zeros(2000)
    tot = 0
    for r in runs:
        print(f'Getting waveforms from run {r}...')
        with h5py.File(f'{raw_dir}/run{r}.lh5') as f:
            ndval = drop_badwfs(f['ORSIS3302DecoderForEnergy']['raw']['waveform']['values'][()])
            subbed = ndval.T - np.amin(ndval, axis=1)
            ndval_norm = (subbed/np.amax(subbed, axis=0)).T
            #print((np.amin(ndval_norm, axis=1) == 0).all(), (np.amax(ndval_norm, axis=1) == 1).all())
            sums += np.sum(ndval_norm, axis=0)
            tot += len(ndval)
    means = sums/tot
    print(f'Sums: {sums}\nMeans: {means}\nTotal: {tot})')
    if plot:
        x = np.arange(2000)
        plt.figure()
        plt.plot(x, means)
        plt.xlabel('sample')
        plt.ylabel('normalized adu')
    return means
                

In [ ]:
run = 4
cycles = files[run][1]
superpulse = get_superpulse(cycles)

In [ ]:
superpulse_down = get_superpulse(np.arange(10,16))

In [ ]:
x = np.arange(1999)
plt.figure()
plt.plot(x, superpulse_down[:-1])
#plt.plot(x, superpulse[:-1])
plt.xlabel('sample')
plt.ylabel('normalized adu')

In [ ]:
run = 4
prefix = files[run][0]
cycles = files[run][1]
browser = WaveformBrowser([f'{raw_dir}/run{cycles[i]}.lh5' for i in np.arange(len(cycles))], 'ORSIS3302DecoderForEnergy/raw')